---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [191]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd
    import numpy as np
    university=pd.read_table('university_towns.txt',delimiter='[', header=None, names= {'Raw Name','New Col'}, index_col=None)
    university['Raw Name'] = university['Raw Name'].str[:-1]
    university['Raw Name'] = university['Raw Name'].replace(to_replace='edit', value=0)
    university['Raw Name'] = university['Raw Name'].replace(to_replace=np.NaN, value=1)
    university['Raw Name'] = university['Raw Name'].convert_objects(convert_numeric=True)
    university.loc[university['Raw Name'] > 0, 'Raw Name'] = 'Region1'
    university.loc[university['Raw Name'] == 0, 'Raw Name'] = 'State'
    x = university.pivot(index=None, columns='Raw Name', values='New Col')
    y = pd.DataFrame(x.to_records())
    z = y[[2,3]]
    z.fillna(method='ffill', inplace=True)
    z['Region'] = z['Region1']
    m = z[[1,2]]
    m['Region'].replace('None', np.nan, inplace=True)
    m['Region'] = m['Region'].replace(to_replace=np.NaN, value=np.NaN)
    n = m.dropna()
    n['Region'] = n['Region'].str.split('(', 1).str[0]
    n.columns=['State','RegionName']
    return n
    
get_list_of_university_towns()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
9,Alaska,Tuskegee
10,Alaska,Fairbanks


In [192]:
def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp=pd.read_excel('gdplev.xls', header=None, skiprows=220, index_col=None, names=None)[[4,6]]
    gdp.columns=['Quarters','Quarterly GDP 2009 Billions']
    gdp['difference'] = gdp['Quarterly GDP 2009 Billions'] - gdp['Quarterly GDP 2009 Billions'].shift(1)
 


    for i in gdp.index:
        if (gdp.iloc[i+1:i+2,2:3] < 0).bool() & (gdp.iloc[i+2:i+3,2:3] < 0).bool():
            return gdp.iloc[i+1]['Quarters']
    return None  
    
get_recession_start()

'2008q3'

In [193]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp=pd.read_excel('gdplev.xls', header=None, skiprows=220, index_col=None, names=None)[[4,6]]
    gdp.columns=['Quarters','Quarterly GDP 2009 Billions']
    gdp['difference'] = gdp['Quarterly GDP 2009 Billions'] - gdp['Quarterly GDP 2009 Billions'].shift(1)
 


    for i in gdp.index:
        if (gdp.iloc[i+1:i+2,2:3] < 0).bool() & (gdp.iloc[i+2:i+3,2:3] < 0).bool():
            x= gdp.iloc[i+1:i+2,2:3]
            if (gdp.iloc[i+3:i+4,2:3] > 0).bool() & (gdp.iloc[i+4:i+5,2:3] > 0).bool():
                return gdp.iloc[i+4]['Quarters']
    return None

get_recession_end()

'2009q4'

In [194]:

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5)
    GDP.drop(GDP.index[:214], inplace = True)
    GDP.dropna(axis = 1, how = 'all', inplace = True)
    GDP = GDP[[0,2]]
    GDP.columns = ["Quarters", 'GDP in billions in current dollars']
    GDP.set_index("Quarters", inplace = True)
    GDP_diff = GDP.diff()
    flag = 0
    for i, quarter in enumerate(GDP_diff.index):
        if (flag == 1) and (GDP_diff.iloc[i] > 0).bool() and (GDP_diff.iloc[i+1] > 0).bool():
            end_idx = i+1
            break
        if (flag == 0) and (GDP_diff.iloc[i] < 0).bool() and (GDP_diff.iloc[i+1] < 0).bool():
            flag = 1
            start_idx = i
    GDP_recession = GDP.iloc[start_idx: end_idx + 1]
    idx = GDP_recession.idxmin()
    return idx[0]
print(get_recession_bottom())

2009q2


In [188]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

Columns: 
 Index(['2000q1', '2000q2', '2000q3', '2000q4', '2001q1', '2001q2', '2001q3',
       '2001q4', '2002q1', '2002q2', '2002q3', '2002q4', '2003q1', '2003q2',
       '2003q3', '2003q4', '2004q1', '2004q2', '2004q3', '2004q4', '2005q1',
       '2005q2', '2005q3', '2005q4', '2006q1', '2006q2', '2006q3', '2006q4',
       '2007q1', '2007q2', '2007q3', '2007q4', '2008q1', '2008q2', '2008q3',
       '2008q4', '2009q1', '2009q2', '2009q3', '2009q4', '2010q1', '2010q2',
       '2010q3', '2010q4', '2011q1', '2011q2', '2011q3', '2011q4', '2012q1',
       '2012q2', '2012q3', '2012q4', '2013q1', '2013q2', '2013q3', '2013q4',
       '2014q1', '2014q2', '2014q3', '2014q4', '2015q1', '2015q2', '2015q3',
       '2015q4', '2016q1', '2016q2', '2016q3'],
      dtype='object')
# Rows:  10730


2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

In [204]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    towns = get_list_of_university_towns()
    startdate = get_recession_start()
    bottomdate = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    
    houses = houses.reset_index()
    houses['recession_diff'] = houses[startdate] - houses[bottomdate]
    
    towns_houses = pd.merge(houses, towns, how='inner', on=['State', 'RegionName'])
    towns_houses['ctown'] = True
    houses = pd.merge(houses, towns_houses, how='outer', on = ['State', 'RegionName',
                                                              bottomdate, startdate, 
                                                              'recession_diff'])
    houses['ctown'] = houses['ctown'].fillna(False)
    unitowns = houses[houses['ctown'] == True]
    not_unitowns = houses[houses['ctown'] == False]
    
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    different = True if p < 0.01 else False
    better = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"
    
            
    return different, p, better
run_ttest()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(False, nan, 'non-university town')